<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/Simulationofanoilstabilizationprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simulation of an oil stabilization process
In this notebook we will establish code for using NeqSim for simulation of an oil stabilization process

In [1]:
# Setting up the Colab page to use neqsim
%%capture
!pip install neqsim
import neqsim
from neqsim.thermo.thermoTools import *

# Process flow diagram

In [2]:
inputdata = {
  'feedFlowRateWell': 10,
  'wellPressure': 180.0,
  'wellTemperature':100.0,
  'topsidePressure': 90.0,
  'topsideTemperature': 5.0,
  'firstStagePressure': 75.0, 
  'temperatureOilHeater' : 75.9,
  'secondStagePressure': 8.6,
  'thirdStagePressure': 1.9,
  'firstStageSuctionCoolerTemperature': 25.3, 
  'secondStageSuctionCoolerTemperature': 24.5, 
  'thirdStageSuctionCoolerTemperature':25.0,
  'firstStageExportCoolerTemperature': 25.3, 
  'secondStageExportCoolerTemperature': 24.5
}

In [3]:
from neqsim.thermo import fluid, printFrame
from neqsim.process import compressor, cooler, separator3phase, getProcess, clearProcess, mixer, heater, stream, pump, separator, runProcess, stream, saturator, valve, filters, heatExchanger, simpleTEGAbsorber,distillationColumn, waterStripperColumn, recycle2, setpoint, calculator

clearProcess()

wellFluid = fluid('pr')
wellFluid.addComponent("nitrogen", 0.08)
wellFluid.addComponent("CO2", 3.56)
wellFluid.addComponent("methane", 87.36)
wellFluid.addComponent("ethane", 4.02)
wellFluid.addComponent("propane", 1.54)
wellFluid.addComponent("i-butane", 0.2)
wellFluid.addComponent("n-butane", 0.42)
wellFluid.addComponent("i-pentane", 0.15)
wellFluid.addComponent("n-pentane", 0.20)
		
wellFluid.addTBPfraction("C6", 3.24, 84.99/1000.0, 695.0/1000.0)
wellFluid.addTBPfraction("C7", 1.34, 97.87/1000.0, 718.0/1000.0)
wellFluid.addTBPfraction("C8", 1.33, 111.54/1000.0, 729.0/1000.0)
wellFluid.addTBPfraction("C9", 1.19, 126.1/1000.0, 749.0/1000.0)
wellFluid.addTBPfraction("C10", 1.15, 140.14/1000.0, 760.0/1000.0)
wellFluid.addTBPfraction("C11", 1.69, 175.0/1000.0, 830.0/1000.0)
wellFluid.addTBPfraction("C12", 1.5, 280.0/1000.0, 914.0/1000.0)
wellFluid.addTBPfraction("C13", 2.103, 560.0/1000.0, 980.0/1000.0)

wellFluid.setMixingRule(2)
wellFluid.init(0)

wellFluid.setMolarComposition([0.08, 3.56, 87.36, 4.02, 1.54, 0.2, 0.42, 0.15, 0.2, 1.24, 1.34, 1.33, 1.19, 1.15, 1.69, 1.5, 1.03])

wellStream = stream(wellFluid)
wellStream.setName("dry well stream")
wellStream.setFlowRate(inputdata['feedFlowRateWell'], "MSm3/day")
wellStream.setTemperature(inputdata['wellTemperature'], "C")
wellStream.setPressure(inputdata['wellPressure'], "bara")

saturatedFeedGas = saturator(wellStream)
saturatedFeedGas.setName("water saturator")

waterSaturatedFeedGas = stream(saturatedFeedGas.getOutStream())
waterSaturatedFeedGas.setName("water saturated feed gas")

feedTPsetter = heater(waterSaturatedFeedGas)
feedTPsetter.setName('inletTP')
feedTPsetter.setOutPressure(inputdata['topsidePressure'], "bara")
feedTPsetter.setOutTemperature(inputdata['topsideTemperature'], "C")

chokeValve = valve(feedTPsetter.getOutStream())
chokeValve.setOutletPressure(inputdata['firstStagePressure'], 'bara')

feedToOffshoreProcess = stream(chokeValve.getOutStream())
feedToOffshoreProcess.setName("feed to offshore")

firstStageSeparator = separator3phase(feedToOffshoreProcess)
firstStageSeparator.setName("1st stage separator")

oilHeaterFromFirstStage = heater(firstStageSeparator.getOilOutStream())
oilHeaterFromFirstStage.setName("oil heater second stage")
oilHeaterFromFirstStage.setOutTemperature(inputdata['temperatureOilHeater'],'C')

oilThrotValve = valve(oilHeaterFromFirstStage.getOutStream())
oilThrotValve.setName("valve oil from first stage")
oilThrotValve.setOutletPressure(inputdata['secondStagePressure'])

secondStageSeparator = separator3phase(oilThrotValve.getOutStream())
secondStageSeparator.setName("2nd stage separator")

oilThrotValve2 = valve(secondStageSeparator.getOilOutStream())
oilThrotValve2.setName("valve oil from second stage")
oilThrotValve2.setOutletPressure(inputdata['thirdStagePressure'])

thirdStageSeparator = separator3phase(oilThrotValve2.getOutStream())
thirdStageSeparator.setName("3rd stage separator")

oilThirdStageToSep =  wellStream.clone()
oilThirdStageToSep.setName("resyc oil")
thirdStageSeparator.addStream(oilThirdStageToSep)

stableOil = stream(thirdStageSeparator.getOilOutStream())
stableOil.setName("stable oil")

stableOilPump = pump(stableOil,15.0,"stable oil pump")

firstStageCooler = cooler(thirdStageSeparator.getGasOutStream())
firstStageCooler.setName("1st stage cooler")
firstStageCooler.setOutTemperature(inputdata['firstStageSuctionCoolerTemperature'],'C')

firstStageScrubber = separator(firstStageCooler.getOutStream())
firstStageScrubber.setName("1st stage scrubber")

firstStageCompressor = compressor(firstStageScrubber.getGasOutStream())
firstStageCompressor.setName("1st stage compressor")
firstStageCompressor.setOutletPressure(inputdata['secondStagePressure'])
firstStageCompressor.setIsentropicEfficiency(0.75)

secondStageCooler = cooler(firstStageCompressor.getOutStream())
secondStageCooler.setName("2nd stage cooler")
secondStageCooler.setOutTemperature(inputdata['secondStageSuctionCoolerTemperature'],'C')

secondStageScrubber = separator(secondStageCooler.getOutStream())
secondStageScrubber.setName("2nd stage scrubber")

secondStageCompressor = compressor(secondStageScrubber.getGasOutStream())
secondStageCompressor.setName("2nd stage compressor")
secondStageCompressor.setOutletPressure(inputdata['firstStagePressure'])
secondStageCompressor.setIsentropicEfficiency(0.75)

richGasMixer = mixer("fourth Stage mixer")
richGasMixer.addStream(secondStageCompressor.getOutStream())
richGasMixer.addStream(firstStageSeparator.getGasOutStream())

dewPointControlCooler = cooler(richGasMixer.getOutStream())
dewPointControlCooler.setName("dew point cooler")
dewPointControlCooler.setOutTemperature(inputdata['thirdStageSuctionCoolerTemperature'],'C')

dewPointScrubber = separator(dewPointControlCooler.getOutStream())
dewPointScrubber.setName("dew point scrubber")

lpLiqmixer = mixer("LP liq gas mixer");
lpLiqmixer.addStream(firstStageScrubber.getLiquidOutStream());
lpLiqmixer.addStream(secondStageScrubber.getLiquidOutStream());
lpLiqmixer.addStream(dewPointScrubber.getLiquidOutStream());

lpResycle = recycle2("LP liq resycle")
lpResycle.addStream(lpLiqmixer.getOutStream())
lpResycle.setOutletStream(oilThirdStageToSep)

exportCompressor1 = compressor(dewPointScrubber.getGasOutStream())
exportCompressor1.setName("export 1st stage")
exportCompressor1.setOutletPressure(140.0)
exportCompressor1.setIsentropicEfficiency(0.75)

exportInterstageCooler = cooler(exportCompressor1.getOutStream())
exportInterstageCooler.setName("interstage stage cooler")
exportInterstageCooler.setOutTemperature(inputdata['firstStageExportCoolerTemperature'],'C')

exportCompressor2= compressor(exportInterstageCooler.getOutStream())
exportCompressor2.setName("export 2nd stage")
exportCompressor2.setOutletPressure(200.0)
exportCompressor2.setIsentropicEfficiency(0.75)

exportCooler = cooler(exportCompressor1.getOutStream())
exportCooler.setName("export cooler")
exportCooler.setOutTemperature(inputdata['secondStageExportCoolerTemperature'],'C')

exportGas = stream(exportCooler.getOutStream())
exportGas.setName("export gas")

oilprocess = getProcess()


# Start simulation as a thread

In [4]:
#oilprocess.run() #run this to finish thread before continuing
thread = oilprocess.runAsThread()
thread.join(50000) # set maximum time to 50000 msec before continuing


# Check if simulation has converged/stopped

True if still running.
False if finished.

In [5]:
thread.isAlive() 

False

# Read results from simulation

In [6]:
TVP = stableOil.TVP(20.0, 'C')
cricondenbar = exportGas.CCB('bara')
powerComp1 = oilprocess.getUnit("1st stage compressor").getPower()/1.0e3
powerComp2 = oilprocess.getUnit("2nd stage compressor").getPower()/1.0e3
powerExpComp1 = oilprocess.getUnit("export 1st stage").getPower()/1.0e3
powerExpComp2 = oilprocess.getUnit("export 2nd stage").getPower()/1.0e3

print('TVP [bara] ', TVP)
print('cricondenbar [bara] ', cricondenbar)
print('recomp power 1 [kW] ', powerComp1)
print('recomp power 2 [kW] ', powerComp2)
print('exp power 1 [kW] ', powerExpComp1)
print('exp power 2 [kW] ', powerExpComp2)


TVP [bara]  1.0021621341864873
cricondenbar [bara]  88.2849574978568
recomp power 1 [kW]  144.1128950584195
recomp power 2 [kW]  185.18376003219316
exp power 1 [kW]  7573.824136763184
exp power 2 [kW]  3916.5695889838867


In [7]:
from neqsim.thermo import fluid, printFrame
printFrame(waterSaturatedFeedGas.getFluid())

| 0                  | 1           | 2          | 3           | 4   | 5   | 6               |
|:-------------------|:------------|:-----------|:------------|:----|:----|:----------------|
|                    | total       | GAS        | OIL         |     |     |                 |
| nitrogen           | 7.33587E-4  | 8.31898E-4 | 2.07073E-4  |     |     | [mole fraction] |
| CO2                | 3.26446E-2  | 3.43896E-2 | 2.3299E-2   |     |     | [mole fraction] |
| methane            | 8.01077E-1  | 8.76413E-1 | 3.97608E-1  |     |     | [mole fraction] |
| ethane             | 3.68627E-2  | 3.72651E-2 | 3.47077E-2  |     |     | [mole fraction] |
| propane            | 1.41215E-2  | 1.30485E-2 | 1.98681E-2  |     |     | [mole fraction] |
| i-butane           | 1.83397E-3  | 1.56455E-3 | 3.27685E-3  |     |     | [mole fraction] |
| n-butane           | 3.85133E-3  | 3.1271E-3  | 7.73001E-3  |     |     | [mole fraction] |
| i-pentane          | 1.37548E-3  | 9.92069E-4 | 3.42884E-3

# Save process to file

In [8]:
oilprocess.save('oilpro.neqsim')

# Sizing of heat exchangers

https://www.engineeringtoolbox.com/heat-transfer-coefficients-exchangers-d_450.html




In [9]:
import math
U_Wm2K = 100.0

water_inlet_temperature = 15.0
water_outlet_temperature = oilprocess.getUnit('1st stage cooler').getInStream().getTemperature("C")-10.0
gas_inlet_temperature=oilprocess.getUnit('1st stage cooler').getInStream().getTemperature("C")
gas_outlet_temperature = oilprocess.getUnit('1st stage cooler').getOutStream().getTemperature("C")


deltaTA = gas_inlet_temperature-water_outlet_temperature
deltaTB = gas_outlet_temperature-water_inlet_temperature

LMTD = (deltaTA-deltaTB)/(math.log(deltaTA/deltaTB))

Q = -oilprocess.getUnit('1st stage cooler').getEnergyInput()
A = Q/U_Wm2K/LMTD
print("LMTD ", round(LMTD,3), " degC")
print("heat exchanger duty ", round(oilprocess.getUnit('1st stage cooler').getEnergyInput()/1e6,3), " MW")
print("heat exchanger area ", round(A,3), " m^2")

cPwater = 4200.0 #J/kgC
densityWater = 1000.0 #kg/m3
waterCirculationRate =  Q/(cPwater*(water_outlet_temperature-water_inlet_temperature))*3600/densityWater
print("water circulation rate ", round(waterCirculationRate,3), " m3/hr")


water_inlet_temperature = 15.0
water_outlet_temperature = oilprocess.getUnit('2nd stage cooler').getInStream().getTemperature("C")-10.0
gas_inlet_temperature=oilprocess.getUnit('2nd stage cooler').getInStream().getTemperature("C")
gas_outlet_temperature = oilprocess.getUnit('2nd stage cooler').getOutStream().getTemperature("C")


deltaTA = gas_inlet_temperature-water_outlet_temperature
deltaTB = gas_outlet_temperature-water_inlet_temperature

LMTD = (deltaTA-deltaTB)/(math.log(deltaTA/deltaTB))
Q = -oilprocess.getUnit('2nd stage cooler').getEnergyInput()
A = Q/U_Wm2K/LMTD
print("LMTD2 ", round(LMTD,3), " degC")
print("heat exchanger duty2 ", round(oilprocess.getUnit('2nd stage cooler').getEnergyInput()/1e6,3), " MW")
print("heat exchanger area2 ", round(A,3), " m^2")

cPwater = 4200.0 #J/kgC
densityWater = 1000.0 #kg/m3
waterCirculationRate =  Q/(cPwater*(water_outlet_temperature-water_inlet_temperature))*3600/densityWater
print("water circulation rate ", round(waterCirculationRate,3), " m3/hr")

water_inlet_temperature = 15.0
water_outlet_temperature = oilprocess.getUnit('interstage stage cooler').getInStream().getTemperature("C")-10.0
gas_inlet_temperature=oilprocess.getUnit('interstage stage cooler').getInStream().getTemperature("C")
gas_outlet_temperature = oilprocess.getUnit('interstage stage cooler').getOutStream().getTemperature("C")


deltaTA = gas_inlet_temperature-water_outlet_temperature
deltaTB = gas_outlet_temperature-water_inlet_temperature

LMTD = (deltaTA-deltaTB)/(math.log(deltaTA/deltaTB))
Q = -oilprocess.getUnit('interstage stage cooler').getEnergyInput()
A = Q/U_Wm2K/LMTD
print("LMTD3 ", round(LMTD,3), " degC")
print("heat exchanger duty3 ", round(oilprocess.getUnit('interstage stage cooler').getEnergyInput()/1e6,3), " MW")
print("heat exchanger area3 ", round(A,3), " m^2")

cPwater = 4200.0 #J/kgC
densityWater = 1000.0 #kg/m3
waterCirculationRate =  Q/(cPwater*(water_outlet_temperature-water_inlet_temperature))*3600/densityWater
print("water circulation rate ", round(waterCirculationRate,3), " m3/hr")

water_inlet_temperature = 15.0
water_outlet_temperature = oilprocess.getUnit('export cooler').getInStream().getTemperature("C")-10.0
gas_inlet_temperature=oilprocess.getUnit('export cooler').getInStream().getTemperature("C")
gas_outlet_temperature = oilprocess.getUnit('export cooler').getOutStream().getTemperature("C")


deltaTA = gas_inlet_temperature-water_outlet_temperature
deltaTB = gas_outlet_temperature-water_inlet_temperature

LMTD = (deltaTA-deltaTB)/(math.log(deltaTA/deltaTB))
Q = -oilprocess.getUnit('export cooler').getEnergyInput()
A = Q/U_Wm2K/LMTD
print("LMTD4 ", round(LMTD,3), " degC")
print("heat exchanger duty4 ", round(oilprocess.getUnit('export cooler').getEnergyInput()/1e6,3), " MW")
print("heat exchanger area4 ", round(A,3), " m^2")

cPwater = 4200.0 #J/kgC
densityWater = 1000.0 #kg/m3
waterCirculationRate =  Q/(cPwater*(water_outlet_temperature-water_inlet_temperature))*3600/densityWater
print("water circulation rate ", round(waterCirculationRate,3), " m3/hr")

LMTD  10.149  degC
heat exchanger duty  -0.144  MW
heat exchanger area  141.873  m^2
water circulation rate  2.876  m3/hr
LMTD2  9.748  degC
heat exchanger duty2  -0.215  MW
heat exchanger area2  220.854  m^2
water circulation rate  2.068  m3/hr
LMTD3  10.149  degC
heat exchanger duty3  -12.325  MW
heat exchanger area3  12143.337  m^2
water circulation rate  182.854  m3/hr
LMTD4  9.748  degC
heat exchanger duty4  -12.511  MW
heat exchanger area4  12834.364  m^2
water circulation rate  185.616  m3/hr


# Sizing of gas scrubbers


In [10]:
gas_load_factor = 0.1
gas_density = oilprocess.getUnit('1st stage scrubber').getGasOutStream().getFluid().getDensity("kg/m3")
oil_density = oilprocess.getUnit('1st stage scrubber').getLiquidOutStream().getFluid().getDensity("kg/m3")
flowRate = oilprocess.getUnit('1st stage scrubber').getGasOutStream().getFluid().getFlowRate("m3/sec")

V_t = gas_load_factor*math.sqrt((oil_density-gas_density)/gas_density)
A_scrub = flowRate/V_t

D_scrub = math.sqrt(A_scrub/(3.14/4.0))

print("gas density ", round(gas_density,3), " [kg/m3)]")
print("oil density ", round(oil_density,3), " [kg/m3]")
print("Maximum gas velocity ", round(V_t,3), " [m/sec]")
print("Minimum scrubber internal diameter ", round(D_scrub,3), " [m]")

gas_load_factor = 0.1
gas_density = oilprocess.getUnit('2nd stage scrubber').getGasOutStream().getFluid().getDensity("kg/m3")
oil_density = oilprocess.getUnit('2nd stage scrubber').getLiquidOutStream().getFluid().getDensity("kg/m3")
flowRate = oilprocess.getUnit('2nd stage scrubber').getGasOutStream().getFluid().getFlowRate("m3/sec")

V_t = gas_load_factor*math.sqrt((oil_density-gas_density)/gas_density)
A_scrub = flowRate/V_t

D_scrub = math.sqrt(A_scrub/(3.14/4.0))

print("gas density ", round(gas_density,3), " [kg/m3)]")
print("oil density ", round(oil_density,3), " [kg/m3]")
print("Maximum gas velocity ", round(V_t,3), " [m/sec]")
print("Minimum scrubber internal diameter ", round(D_scrub,3), " [m]")

gas_load_factor = 0.1
gas_density = oilprocess.getUnit('dew point scrubber').getGasOutStream().getFluid().getDensity("kg/m3")
oil_density = oilprocess.getUnit('dew point scrubber').getLiquidOutStream().getFluid().getDensity("kg/m3")
if(abs(oil_density-gas_density)<10.0):
  oil_density = 800.0
flowRate = oilprocess.getUnit('dew point scrubber').getGasOutStream().getFluid().getFlowRate("m3/sec")

V_t = gas_load_factor*math.sqrt((oil_density-gas_density)/gas_density)
A_scrub = flowRate/V_t

D_scrub = math.sqrt(A_scrub/(3.14/4.0))

print("gas density ", round(gas_density,3), " [kg/m3)]")
print("oil density ", round(oil_density,3), " [kg/m3]")
print("Maximum gas velocity ", round(V_t,3), " [m/sec]")
print("Minimum scrubber internal diameter ", round(D_scrub,3), " [m]")

gas density  2.631  [kg/m3)]
oil density  691.741  [kg/m3]
Maximum gas velocity  1.618  [m/sec]
Minimum scrubber internal diameter  0.519  [m]
gas density  11.229  [kg/m3)]
oil density  621.307  [kg/m3]
Maximum gas velocity  0.737  [m/sec]
Minimum scrubber internal diameter  0.336  [m]
gas density  64.14  [kg/m3)]
oil density  439.968  [kg/m3]
Maximum gas velocity  0.242  [m/sec]
Minimum scrubber internal diameter  2.431  [m]


In [11]:
printFrame(oilprocess.getUnit('dew point scrubber').getFluid())

| 0                  | 1          | 2          | 3   | 4   | 5   | 6               |
|:-------------------|:-----------|:-----------|:----|:----|:----|:----------------|
|                    | total      | GAS        |     |     |     |                 |
| nitrogen           | 8.8267E-4  | 8.8267E-4  |     |     |     | [mole fraction] |
| CO2                | 3.3853E-2  | 3.3853E-2  |     |     |     | [mole fraction] |
| methane            | 9.16928E-1 | 9.16928E-1 |     |     |     | [mole fraction] |
| ethane             | 3.44028E-2 | 3.44028E-2 |     |     |     | [mole fraction] |
| propane            | 9.56489E-3 | 9.56489E-3 |     |     |     | [mole fraction] |
| i-butane           | 8.83262E-4 | 8.83262E-4 |     |     |     | [mole fraction] |
| n-butane           | 1.48447E-3 | 1.48447E-3 |     |     |     | [mole fraction] |
| i-pentane          | 2.94465E-4 | 2.94465E-4 |     |     |     | [mole fraction] |
| n-pentane          | 3.1021E-4  | 3.1021E-4  |     |     |     